# Load data

In [ ]:
from tqdm.notebook import trange
import numpy as np
N = 60000

In [ ]:
text_arrays = np.array_split(np.fromfile("../../Downloads/text_arrays_swaes_60k.bin", dtype=np.uint8), N)

In [ ]:
key_arrays = np.array_split(np.fromfile("../../Downloads/key_arrays_swaes_60k.bin", dtype=np.uint8), N)

In [ ]:
traces_enc = np.array(np.array_split(np.fromfile("../../Downloads/traces_enc_swaes_60k.bin", dtype=np.float64), N))

# SBox

In [ ]:
sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess]

HW = [bin(n).count("1") for n in range(0, 256)]

# CPA Attack -- Method 1

In [ ]:
numtraces = len(traces_enc_full) #total number of traces
numpoint = len(traces_enc_full[0]) #samples per trace

pt = text_arrays
knownkey = key_arrays[0]

key_guess = []
full_cpa_list = []
known_key = [0x54, 0xee, 0xff, 0x11, 0x21, 0x32, 0xeb, 0x43, 0xbb, 0xaa, 0xff, 0x23, 0x98, 0x54, 0x12, 0x12]

for bnum in trange(0, 16):
    cpaoutput = [0] * 256
    maxcpa = [0] * 256
    for kguess in trange(0, 256):

        # Initialize arrays &amp; variables to zero
        sumnum = np.zeros(numpoint)
        sumden1 = np.zeros(numpoint)
        sumden2 = np.zeros(numpoint)
        
        hyp = np.zeros(numtraces)
        for tnum in range(0, numtraces):
            hyp[tnum] = HW[intermediate(pt[tnum][bnum], kguess)]

        # Mean of hypothesis
        meanh = np.mean(hyp, dtype=np.float64)

        # Mean of all points in trace
        meant = np.mean(traces_enc, axis=0, dtype=np.float64)

        # For each trace, do the following
        for tnum in range(0, numtraces):
            hdiff = (hyp[tnum] - meanh)
            tdiff = traces_enc[tnum, :] - meant

            sumnum = sumnum + (hdiff * tdiff)
            sumden1 = sumden1 + hdiff * hdiff
            sumden2 = sumden2 + tdiff * tdiff

        cpaoutput[kguess] = sumnum / np.sqrt(sumden1 * sumden2)
        maxcpa[kguess] = max(abs(cpaoutput[kguess]))

    full_cpa_list.append(cpaoutput)
    np.array(full_cpa_list[bnum]).tofile("new_cpa_byte%2d.bin" %(bnum))
    sorted_args = np.argsort(maxcpa)[::-1]
    key_guess.append(sorted_args[0])
    print("Subkey %2d - most likely %02X (actual %02X)"%(bnum, key_guess[bnum], known_key[bnum]))
    print(" Top 5 guesses: ")
    for i in range(0, 5):
        g = sorted_args[i]
        print("   %02X - Diff = %f"%(g, maxcpa[g]))
    print("\n")

# CPA Attack -- Method 2

In [ ]:
def intermediate(pt, keyguess):
    return sbox[np.uint64(pt) ^ np.uint64(keyguess)]

def mean(X):
    return np.sum(X, axis=0)/len(X)

def std_dev(X, X_bar):
    return np.sqrt(np.sum((X-X_bar)**2, axis=0))

def cov(X, X_bar, Y, Y_bar):
    return np.sum((X-X_bar)*(Y-Y_bar), axis=0)

HW = [bin(n).count("1") for n in range(0, 256)]

In [ ]:
t_bar = mean(np.array(traces_enc))
o_t = std_dev(np.array(traces_enc), t_bar)

key_guess = []
full_cpa_list = []
known_key = [0x54, 0xee, 0xff, 0x11, 0x21, 0x32, 0xeb, 0x43, 0xbb, 0xaa, 0xff, 0x23, 0x98, 0x54, 0x12, 0x12]

for bnum in trange(0, 16):
    max_cpa = [0] * 256
    byte_cpa_list = [0] * 256
    for kguess in trange(0, 256):
        hws = np.array([[HW[intermediate(textin[bnum],kguess)] for textin in text_arrays]]).transpose()
        hws_bar = mean(hws)
        o_hws = std_dev(hws, hws_bar)
        correlation = cov(np.array(traces_enc), t_bar, hws, hws_bar)
        cpaoutput = correlation/(o_t*o_hws)
        byte_cpa_list[kguess] = cpaoutput
        max_cpa[kguess] = np.max(abs(cpaoutput))
        
    full_cpa_list.append(byte_cpa_list[:])
    np.array(full_cpa_list[bnum]).tofile("cpa_byte%2d.bin" %(bnum))
    sorted_args = np.argsort(max_cpa)[::-1]
    key_guess.append(sorted_args[0])
    print("Subkey %2d - most likely %02X (actual %02X)"%(bnum, key_guess[bnum], known_key[bnum]))
    print(" Top 5 guesses: ")
    for i in range(0, 5):
        g = sorted_args[i]
        print("   %02X - Diff = %f"%(g, max_cpa[g]))
    print("\n")

# Plot

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pylab as plt
correct_key = 84 #0x54
subkey = 0
plt.figure()
for guess in range(0, 256):
    if (guess == correct_key):
        plt.plot(full_cpa_list[subkey][guess],color="red",linewidth=1.5, linestyle='-',zorder=255)
    else:
        plt.plot(full_cpa_list[subkey][guess],color="grey")
plt.show()

# How Many Traces Needed for Method 1

In [ ]:
traces_enc_full = np.array(np.array_split(np.fromfile("../../Downloads/traces_enc_swaes_60k.bin", dtype=np.float64), N))

In [ ]:
def num_same(a, b):
    
    if len(a) != len(b):
        raise ValueError("Arrays must be same length!")
    
    #Count how many list items match up
    same = 0
    for i, _ in enumerate(a):
        if a[i] == b[i]:
            same += 1
    
    return same

In [ ]:
full_cpa_list = []
key_full_guess = []
how_many_correct_keybytes = []
is_same = 0

knownkey = key_arrays[0]
traces_enc = []
pt = []
known_key = [0x54, 0xee, 0xff, 0x11, 0x21, 0x32, 0xeb, 0x43, 0xbb, 0xaa, 0xff, 0x23, 0x98, 0x54, 0x12, 0x12]

for traces in trange(1000,25000,1000):
    traces_enc = np.array(traces_enc_full[0:traces])
    pt = np.array(text_arrays[0:traces])
    numtraces = len(traces_enc) 
    numpoint = len(traces_enc[0]) 
    
    key_guess = []
    cpa_list = []
    for bnum in trange(0, 16):
        
        cpaoutput = [0] * 256
        maxcpa = [0] * 256
        
        for kguess in range(0, 256):

        # Initialize arrays &amp; variables to zero
            sumnum = np.zeros(numpoint)
            sumden1 = np.zeros(numpoint)
            sumden2 = np.zeros(numpoint)
        
            hyp = np.zeros(numtraces)
            for tnum in range(0, numtraces):
                hyp[tnum] = HW[intermediate(pt[tnum][bnum], kguess)]

            # Mean of hypothesis
            meanh = np.mean(hyp, dtype=np.float64)

            # Mean of all points in trace
            meant = np.mean(traces_enc, axis=0, dtype=np.float64)

            # For each trace, do the following
            for tnum in range(0, numtraces):
                hdiff = (hyp[tnum] - meanh)
                tdiff = traces_enc[tnum, :] - meant

                sumnum = sumnum + (hdiff * tdiff)
                sumden1 = sumden1 + hdiff * hdiff
                sumden2 = sumden2 + tdiff * tdiff

            cpaoutput[kguess] = sumnum / np.sqrt(sumden1 * sumden2)
            maxcpa[kguess] = max(abs(cpaoutput[kguess]))

        cpa_list.append(cpaoutput)
        sorted_args = np.argsort(maxcpa)[::-1]
        key_guess.append(sorted_args[0])

    full_cpa_list.append(cpa_list)
    key_full_guess.append(key_guess)
    is_same = num_same(key_guess,known_key)
    how_many_correct_keybytes.append(is_same)
    
    print("For %2d traces, correct key byte are %2d" %(traces,is_same))
    print("\n")

# How Many Traces Needed for Method 2

In [ ]:
full_cpa_list = []
key_full_guess = []
how_many_correct_keybytes = []
is_same = 0

traces_enc = []
text_arrays = []

known_key = [0x54, 0xee, 0xff, 0x11, 0x21, 0x32, 0xeb, 0x43, 0xbb, 0xaa, 0xff, 0x23, 0x98, 0x54, 0x12, 0x12]

for traces in trange(1000,25000,1000):
    traces_enc = np.array(traces_enc_full[0:traces])
    text_arrays = np.array(text_arrays_full[0:traces])
    
    t_bar = mean(traces_enc)
    o_t = std_dev(traces_enc, t_bar)
    
    key_guess = []
    cpa_list = []
    for bnum in trange(0, 16):
        
        max_cpa = [0] * 256
        byte_cpa_list = [0] * 256
        
        for kguess in range(0, 256):
            hws = np.array([[HW[intermediate(textin[bnum],kguess)] for textin in text_arrays]]).transpose()
            hws_bar = mean(hws)
            o_hws = std_dev(hws, hws_bar)
            correlation = cov(traces_enc, t_bar, hws, hws_bar)
            cpaoutput = correlation/(o_t*o_hws)
            byte_cpa_list[kguess] = cpaoutput
            max_cpa[kguess] = np.max(abs(cpaoutput))
            
        cpa_list.append(byte_cpa_list)
        sorted_args = np.argsort(max_cpa)[::-1]
        key_guess.append(sorted_args[0])

    full_cpa_list.append(cpa_list)
    key_full_guess.append(key_guess)
    is_same = num_same(key_guess,known_key)
    how_many_correct_keybytes.append(is_same)
    
    print("For %2d traces, correct key byte are %2d" %(traces,is_same))
    print("\n")